Copyright 2022 data cybernetics ssc GmbH.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import numpy as np
import pandas as pd
import qiskit
from q_alchemy import Client
from q_alchemy.models import Strategy

# Creating a Client

You can easily create a client as soon as you have an API key. You can get yours for free at https://portal.q-alchemy.com! Once you have that, please simply add it to the client constructor parameter `api_key`.

The client connects to the Q-Alchemy Hypermedia+Siren API. You can check it out also at

https://hypermedia-ui-demo.q-alchemy.com/hui?apiPath=https:%2F%2Fjobs.api.q-alchemy.com%2Fapi%2FEntryPoint

where you can visually play around with it. The same methods are also available with this SDK!

In [2]:
# Creating a client is really simple: just add your API key!
client = Client(api_key="<get your own at https://portal.q-alchemy.com>")
root = client.get_jobs_root()
root

JobsRoot(https://jobs.api.q-alchemy.com/api/Jobs)

# Create and Configure a new Job

In [3]:
# Let us create a new and empty job, but rename it right away!
job = root.create_job()
job.rename("Carsten's Test-Job")
job

Job(Created, https://jobs.api.q-alchemy.com/api/Jobs/Job/3600916e-9a15-4131-8f2b-4e9f5ea35233)

In [4]:
# First, let us configure the job with the job's config
# for that we summon up the config resource!
config = job.get_config()
config

Config(Missing, https://jobs.api.q-alchemy.com/api/Jobs/Job/3600916e-9a15-4131-8f2b-4e9f5ea35233/Config)

In [5]:
# Create a job config with the fluent syntax:
config.create_config() \
    .with_use_low_rank(True) \
    .with_max_fidelity_loss(0.1) \
    .with_strategy(Strategy.GREEDY) \
    .with_tags("Test Job", "Q-Alchemy") \
    .upload()

#Check out the job's config as it has been configured!
job_config = config.job_config()
job_config

JobConfig(max_fidelity_loss=0.1, strategy='Greedy', max_combination_size=0, use_low_rank=True, load_per_cyle=5000, secede_modulo=1, max_time_sec=1200, max_loops=200, max_nodes=50000, max_level=2, batch_size=100, log_level='INFO', redis_ttl_seconds=3600, tags=['Test Job', 'Q-Alchemy'])

In [6]:
# Now prepare to load a state vector. We support numpy arrays natively,
# but under the hood pyarrow with parquet is used:
vector = np.load("../tests/data/test_baa_state.12.1.npy")
vector.shape

(4096,)

In [7]:
# Upload the state vector now:
state_vector = job.get_state_vector()
state_vector.upload_vector(vector)

# Check out, what the state actually is that you just uploaded:
downloaded_vector = state_vector.get_vector()
downloaded_vector.shape

(4096,)

In [9]:
# Let's see, what the API found out about our state
state_vector

StateVector(~9 qb, 12 qb, 54669 bytes, https://jobs.api.q-alchemy.com/api/Jobs/Job/3600916e-9a15-4131-8f2b-4e9f5ea35233/StateVector)

In [10]:
# The Job should be ready for processing now, let's see if it is:
job.update().state

'ReadyForProcessing'

In [11]:
# Since we are ready for processing, how about we start it?
job.schedule()

True

In [14]:
print(job.update().state)
pd.DataFrame.from_records(job.get_result().get_result_evolution())

Completed


,TimeElapsedSeconds,FidelityLoss,CxGatesSaved,CxGates,Level,Number
0,0.000000,0.000000,0.0,0.0,0.0,-1.0
1,2.607898,0.083916,2179.0,0.0,1.0,2.0
2,4.181702,0.083916,2179.0,0.0,1.0,2.0


In [18]:
nodes = job.get_result().get_result_nodes()
qc = nodes[0].to_circuit()
qiskit.transpile(qc, basis_gates=["rx", "ry", "rz", "cx", "id"], optimization_level=0).depth(lambda v: v[0].name == "cx")

1377

In [19]:
qc = nodes[1].to_circuit()
qiskit.transpile(qc, basis_gates=["rx", "ry", "rz", "cx", "id"], optimization_level=0).depth(lambda v: v[0].name == "cx")

503